In [ ]:
from util import load_dataset, get_country_dict, get_local_stores
from algorithms import nearest_neighbour_tsp

In [ ]:
df = load_dataset()
country_dict = get_country_dict(df)
netherlands_df = get_local_stores(df, country_code="MY")

In [ ]:
from geopy import distance as geopy_dist

def build_distance_matrix(local_store_df):
    
    n = len(local_store_df)
    distance_mat = [[0 for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        for j in range(i + 1):
            if i == j: continue
            
            latlong_i = (local_store_df.iloc[i]["latitude"], local_store_df.iloc[i]["longitude"])
            latlong_j = (local_store_df.iloc[j]["latitude"], local_store_df.iloc[j]["longitude"])
              
            geodesic_dist = geopy_dist.geodesic(latlong_i, latlong_j).km 
            distance_mat[i][j] = distance_mat[j][i] = geodesic_dist
    
    return distance_mat

In [ ]:
distance_mat_nl = build_distance_matrix(netherlands_df)

In [ ]:
nearest_neighbour_tsp(distance_mat_nl, center_idx=0)

In [ ]:
import os
import warnings
from mapbox import Directions
from collections import OrderedDict

def fetch_delivery_routes(center_idx, path, distance_mat, local_store_df):
    with open("./mapbox_api_key.txt") as f:
        api_key = f.readline()
    os.environ['MAPBOX_ACCESS_TOKEN'] = api_key
    service = Directions()
    
    start_from = path.index(center_idx)
    cycle = path[start_from:] + path[0:start_from + 1]
    
    path_dict = OrderedDict()
    
    for i in range(len(cycle)):
        if i == len(cycle) - 1: break
        
        origin_idx = cycle[i]
        dest_idx = cycle[i + 1]
        
        latlong_origin = (local_store_df.iloc[origin_idx]["latitude"], local_store_df.iloc[origin_idx]["longitude"])
        latlong_dest = (local_store_df.iloc[dest_idx]["latitude"], local_store_df.iloc[dest_idx]["longitude"])
        
        response = service.directions([latlong_origin[::-1], latlong_dest[::-1]], profile="mapbox/driving")
        
        if response.status_code != 200: raise Exception("Failed to retrieve routes from Mapbox API")
        
        response_json = response.json()
        
        if response_json["code"] == "NoRoute":
            warnings.warn("No route found, use geodesic distance")
            cur_dist = distance_mat[origin_idx][dest_idx]
            path_dict[(origin_idx, dest_idx)] = {"distance": cur_dist, "duration": None, "path": None, "code": "NoRoute"}
        else:
            cur_dist = response_json["routes"][0]["distance"] / 1000        # km
            cur_path = response_json["routes"][0]["geometry"]           # polyline string
            cur_dur = response_json["routes"][0]["duration"] / 60  # minutes
            path_dict[(origin_idx, dest_idx)] = {"distance": cur_dist, "duration": cur_dur, "path": cur_path, "code": "Ok"}
    
    return path_dict
        

In [ ]:
import folium
import math
import json
import requests
import polyline

def build_map(center_idx, path, distance_mat, local_store_df):
    def construct_address(store_info):
        address = ""
        for i, key in enumerate(('street_address', 'zip_code', 'city', 'country')):
            try:
                isnan = math.isnan(store_info[key])
            except:
                isnan = False # If input to math.isnan is not a float
            if store_info[key] != "0" and not isnan:
                address += f"{store_info[key]}, " if i != 3 else f"{store_info[key]}."
        return address
    
    def construct_openhours(store_info):
        try:
            isnan = math.isnan(store_info["open_hours"])
        except:
            isnan = False # If input to math.isnan is not a float
        
        if not isnan:
            openhours = {}
            for openhour in store_info["open_hours"].split(", "):
                day, time = openhour.split(" : ")
                openhours[day] = time
            return openhours
        return None
    
    def construct_store_tooltip(store_info, storetype):
        address = construct_address(store_info)
        openhours = construct_openhours(store_info)
        storename = store_info["name"]
        url = store_info["url"]
        tooltip = f"""
            <p><i>{storetype}</i></p>
            <h3>{storename}</h3>
            <hr>
            <p><b>Address</b> : {address}</p>
            <table>
                <tr><th>Opening Hours</th></tr>
                {"".join(f"<tr><td>{day}</td><td>{openhours[day]}</td></tr>" for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] if day in openhours) if openhours != None else f"<tr><td>{'Not available'}</td><td></td></tr>"}
            </table>
            <br>
            <p><b>Check us out on :</b><br><a href="{url}">{url}</a></p>
        """
        return tooltip
    
    def fetch_country_geojson(country_name):
        res = requests.get(f"https://nominatim.openstreetmap.org/search?country={country_name}&polygon_geojson=1&format=json")
        country_geojson = json.loads(res.content.decode())[0]["geojson"]
        return country_geojson
    
    folium_map = folium.Map(location=(local_store_df.iloc[center_idx]["latitude"], local_store_df.iloc[center_idx]["longitude"]), control_scale=True)
    folium.GeoJson(fetch_country_geojson(country_name=local_store_df.iloc[0]["country"]), name=local_store_df.iloc[0]["country"], style_function=lambda x: {'fillOpacity': 0.1}, show=False).add_to(folium_map)
    
    for idx, store_info in local_store_df.iterrows():
        latlong_store = (store_info["latitude"], store_info["longitude"])
        if idx == center_idx:
            center_tooltip = construct_store_tooltip(store_info, storetype="Moonbucks' Distribution Center")
            folium.Marker(location=latlong_store, popup=folium.Popup(center_tooltip, min_width=300, max_width=300), tooltip=folium.Tooltip(center_tooltip), icon=folium.Icon(color="red", icon="building-o", prefix="fa")).add_to(folium_map)
        else:
            store_tooltip = construct_store_tooltip(store_info, storetype="Moonbucks' Branch")
            folium.Marker(location=latlong_store, popup=folium.Popup(store_tooltip, min_width=300, max_width=300), tooltip=folium.Tooltip(store_tooltip), icon=folium.Icon(color="green", icon="home", prefix="glyphicon")).add_to(folium_map)
    
    path_dict = fetch_delivery_routes(center_idx, path, distance_mat, local_store_df)
    
    for idx, key in enumerate(path_dict):
        origin_idx, dest_idx = key
        if path_dict[key]["code"] == "NoRoute":
            cur_distance = path_dict[key]["distance"]

            path_popup = f"""
                <h5>From : <i><u>{local_store_df.iloc[origin_idx]["name"]}</u></i> <br>To : <i><u>{local_store_df.iloc[dest_idx]["name"]}</u></i></h5>
                <p style='color: red'><b>No driving route found, geodesic distance is used</b></p>
                <b>Distance : </b> {round(cur_distance, 2)} km <br>
                <b>Duration : </b> -
            """
            latlong_origin = (local_store_df.iloc[origin_idx]["latitude"], local_store_df.iloc[origin_idx]["longitude"])
            latlong_dest = (local_store_df.iloc[dest_idx]["latitude"], local_store_df.iloc[dest_idx]["longitude"])
            
            folium.PolyLine([latlong_origin, latlong_dest], color="green", name=f"{idx + 1} : {local_store_df.iloc[origin_idx]['name']} → {local_store_df.iloc[dest_idx]['name']}").add_child(folium.Popup(path_popup, max_width=300)).add_to(folium_map)
        else:
            cur_duration = path_dict[key]["duration"]
            cur_distance = path_dict[key]["distance"]
            cur_path_coords = polyline.decode(path_dict[key]["path"], geojson=True)
            cur_path_geojson = {"type": "LineString", "coordinates": cur_path_coords}

            path_popup = f"""
                <h5>From : <i><u>{local_store_df.iloc[origin_idx]["name"]}</u></i> <br>To : <i><u>{local_store_df.iloc[dest_idx]["name"]}</u></i></h5>
                <b>Distance : </b> {round(cur_distance, 2)} km <br>
                <b>Duration : </b> {round(cur_duration, 2)} minutes
            """
            folium.GeoJson(cur_path_geojson, style_function=lambda x: {'color': "green"}, name=f"{idx + 1} : {local_store_df.iloc[origin_idx]['name']} → {local_store_df.iloc[dest_idx]['name']}").add_child(folium.Popup(path_popup, max_width=300)).add_to(folium_map)
    
    folium.LayerControl().add_to(folium_map)
    return folium_map

In [ ]:
folium_map = build_map(center_idx=0, path=list(range(len(netherlands_df))), distance_mat=distance_mat_nl, local_store_df=netherlands_df)

In [ ]:
folium_map